In [1]:
import __init__
#
from IPython.display import HTML, display
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
#
# some functions
#
def text_display(text, font_size):
    display(HTML('<font size=%d>' % font_size + text + '</font>'))

def table_display(table_data):
    display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in table_data)
        )
    ))

# Questions
* Q1: What was changed after information boards were set up?
    * H1: Queueing time at the airport in 2009 and 2010 is different
    * H2: Economic profit in 2009 and 2010 is different
    * H3: Productivity in 2009 and 2010 is different

* Q2: Is the impact of information boards valid?
    * Simple regression
    * Multivariate regression
    
# Variables
* All values are total ones during a certain period
* AP trip represents trips which depart from the airport

| Variable        | Description |
| ---------------- |------------------|
| tripNumber       | The number of trips |
| operatingHour    | The operating hour (unit hour) |
| Fare             | The amount of fare (unit S$\$$) |
| apNumber         | The number of AP trips |
| apInNumber       | The number of AP trips whose previous trip ended at the airport |
| apOutNumber      | The difference between apNumber and apInNumber |
| apDuration       | The amount of time passengers are on board (unit minute) |
| apQTime          | The amount of waiting time for taking passengers (unit minute) |
| apFare           | The amount of fare earned through AP trips (unit S$\$$) |
| apEconomicProfit | The difference between apFare and opportunity cost (unit S$\$$) |

# Derive variables for analysis
| Variable             | Description |
| --------------------- |------------------|
| QTime/apTrip          | The average queuing time at the airport (unit minute) |
| economicProfit/apTrip | The average economic profit about a AP trip (unit S$\$$) |
| Productivity          | The rate of fare per operating hour (unit S$\$$ / hour) |
| apProductivity        | The rate of apFare per the sum of apDuration and apQTime (unit S$\$$ / hour) |

In [2]:
from information_boards import ssDriversStatisticsMonthBased2009_ap_fpath
from information_boards import ssDriversStatisticsMonthBased2010_ap_fpath

Y2009_df = pd.read_csv(ssDriversStatisticsMonthBased2009_ap_fpath)
Y2010_df = pd.read_csv(ssDriversStatisticsMonthBased2010_ap_fpath)

In [3]:
# Models
dep_v = 'QTime/apTrip'
ib_impact = ['apInRatio']
cv1 = ['tripNumber', 'apNumber', 'Productivity']
learning_variables = ['month', 'month^2']
m1_inDepV = ib_impact
m2a_inDepV = ib_impact + cv1
m2b_inDepV = ib_impact + cv1 + ['economicProfit/apTrip']
m2c_inDepV = ib_impact + cv1 + ['apProductivity']
m2d_inDepV = ib_impact + cv1 + ['economicProfit/apTrip', 'apProductivity']
m3_inDepV = ib_impact + cv1 + ['economicProfit/apTrip', 'apProductivity'] + learning_variables

In [4]:
# M1
dataset = Y2009_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m1_inDepV})
model

/Users/JerryHan88/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://statsmodels.sourceforge.net/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <apInRatio> + <intercept>

Number of Observations:         6567
Number of Degrees of Freedom:   2

R-squared:         0.0000
Adj R-squared:    -0.0002

Rmse:              8.3239

F-stat (1, 6565):     0.0002, p-value:     0.9892

Degrees of Freedom: model 1, resid 6565

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
     apInRatio    -0.0045     0.3362      -0.01     0.9892    -0.6635     0.6544
     intercept    37.4242     0.2216     168.87     0.0000    36.9899    37.8586
---------------------------------End of Summary---------------------------------

In [5]:
# M1
dataset = Y2010_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m1_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <apInRatio> + <intercept>

Number of Observations:         8272
Number of Degrees of Freedom:   2

R-squared:         0.0144
Adj R-squared:     0.0142

Rmse:              7.3336

F-stat (1, 8270):   120.5522, p-value:     0.0000

Degrees of Freedom: model 1, resid 8270

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
     apInRatio    -3.0166     0.2747     -10.98     0.0000    -3.5551    -2.4781
     intercept    33.3700     0.1940     172.05     0.0000    32.9899    33.7502
---------------------------------End of Summary---------------------------------

In [6]:
# M2a
dataset = Y2009_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m2a_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <tripNumber>
             + <intercept>

Number of Observations:         6567
Number of Degrees of Freedom:   5

R-squared:         0.0325
Adj R-squared:     0.0319

Rmse:              8.1893

F-stat (4, 6562):    55.1218, p-value:     0.0000

Degrees of Freedom: model 4, resid 6562

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity    -0.3917     0.0290     -13.52     0.0000    -0.4485    -0.3349
     apInRatio     1.3354     0.4460       2.99     0.0028     0.4612     2.2096
      apNumber    -0.0418     0.0180      -2.32     0.0203    -0.0771    -0.0065
    tripNumber     0.0080     0.0013       6.06     0.0000     0.0054     0.0106
     int

In [7]:
# M2
dataset = Y2010_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m2a_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <tripNumber>
             + <intercept>

Number of Observations:         8272
Number of Degrees of Freedom:   5

R-squared:         0.0519
Adj R-squared:     0.0515

Rmse:              7.1938

F-stat (4, 8267):   113.1825, p-value:     0.0000

Degrees of Freedom: model 4, resid 8267

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity    -0.3911     0.0224     -17.45     0.0000    -0.4350    -0.3472
     apInRatio    -2.1083     0.3709      -5.68     0.0000    -2.8352    -1.3815
      apNumber    -0.0203     0.0137      -1.48     0.1386    -0.0471     0.0066
    tripNumber     0.0043     0.0010       4.36     0.0000     0.0024     0.0063
     int

In [8]:
# M2b
dataset = Y2009_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m2b_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <economicProfit/apTrip>
             + <tripNumber> + <intercept>

Number of Observations:         6567
Number of Degrees of Freedom:   6

R-squared:         0.7166
Adj R-squared:     0.7164

Rmse:              4.4327

F-stat (5, 6561):  3317.7381, p-value:     0.0000

Degrees of Freedom: model 5, resid 6561

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity    -0.0740     0.0159      -4.66     0.0000    -0.1051    -0.0428
     apInRatio    -2.6645     0.2435     -10.94     0.0000    -3.1417    -2.1872
      apNumber    -0.0488     0.0097      -5.01     0.0000    -0.0679    -0.0297
economicProfit/apTrip    -2.1230     0.0169    -125.84     0.00

In [9]:
# M2b
dataset = Y2010_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m2b_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <economicProfit/apTrip>
             + <tripNumber> + <intercept>

Number of Observations:         8272
Number of Degrees of Freedom:   6

R-squared:         0.6505
Adj R-squared:     0.6503

Rmse:              4.3679

F-stat (5, 8266):  3077.3098, p-value:     0.0000

Degrees of Freedom: model 5, resid 8266

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity    -0.0772     0.0139      -5.57     0.0000    -0.1043    -0.0500
     apInRatio    -4.2691     0.2259     -18.90     0.0000    -4.7119    -3.8264
      apNumber    -0.0348     0.0083      -4.19     0.0000    -0.0511    -0.0185
economicProfit/apTrip    -1.9139     0.0161    -118.99     0.00

In [10]:
# M2c
dataset = Y2009_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m2c_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <apProductivity>
             + <tripNumber> + <intercept>

Number of Observations:         6567
Number of Degrees of Freedom:   6

R-squared:         0.6615
Adj R-squared:     0.6612

Rmse:              4.8444

F-stat (5, 6561):  2564.2810, p-value:     0.0000

Degrees of Freedom: model 5, resid 6561

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity    -0.0460     0.0174      -2.64     0.0084    -0.0801    -0.0118
     apInRatio    -1.7985     0.2654      -6.78     0.0000    -2.3187    -1.2784
      apNumber    -0.0123     0.0107      -1.15     0.2496    -0.0332     0.0086
apProductivity    -2.0239     0.0183    -110.41     0.0000    -2.0598 

In [11]:
# M2c
dataset = Y2010_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m2c_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <apProductivity>
             + <tripNumber> + <intercept>

Number of Observations:         8272
Number of Degrees of Freedom:   6

R-squared:         0.6419
Adj R-squared:     0.6417

Rmse:              4.4216

F-stat (5, 8266):  2963.1343, p-value:     0.0000

Degrees of Freedom: model 5, resid 8266

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity    -0.0020     0.0142      -0.14     0.8889    -0.0298     0.0258
     apInRatio    -4.0513     0.2286     -17.73     0.0000    -4.4993    -3.6034
      apNumber     0.0040     0.0084       0.48     0.6304    -0.0124     0.0205
apProductivity    -1.7336     0.0149    -116.69     0.0000    -1.7627 

In [14]:
# M3
dataset = Y2009_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m3_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <apProductivity>
             + <economicProfit/apTrip> + <month> + <month^2> + <tripNumber>
             + <intercept>

Number of Observations:         6567
Number of Degrees of Freedom:   9

R-squared:         0.7446
Adj R-squared:     0.7443

Rmse:              4.2092

F-stat (8, 6558):  2389.4414, p-value:     0.0000

Degrees of Freedom: model 8, resid 6558

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity    -0.0277     0.0152      -1.83     0.0678    -0.0575     0.0020
     apInRatio    -2.3745     0.2319     -10.24     0.0000    -2.8289    -1.9200
      apNumber    -0.0268     0.0093      -2.88     0.0040    -0.0450    -0.0086
apProduct

In [15]:
# M3
dataset = Y2010_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m3_inDepV})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <apProductivity>
             + <economicProfit/apTrip> + <month> + <month^2> + <tripNumber>
             + <intercept>

Number of Observations:         8272
Number of Degrees of Freedom:   9

R-squared:         0.6934
Adj R-squared:     0.6931

Rmse:              4.0919

F-stat (8, 8263):  2336.0499, p-value:     0.0000

Degrees of Freedom: model 8, resid 8263

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
  Productivity     0.0033     0.0132       0.25     0.8046    -0.0227     0.0292
     apInRatio    -4.1548     0.2120     -19.60     0.0000    -4.5704    -3.7393
      apNumber    -0.0097     0.0078      -1.24     0.2157    -0.0250     0.0057
apProduct

In [16]:
Y2009_drivers = [str(did) for did in set(Y2009_df['driverID'])]
for did in Y2009_drivers:
    Y2009_df[did] = np.where(Y2009_df['driverID'] == int(did), 1, 0)
Y2010_drivers = [str(did) for did in set(Y2010_df['driverID'])]
for did in Y2010_drivers:
    Y2010_df[did] = np.where(Y2010_df['driverID'] == int(did), 1, 0)
m4_inDepV_Y2009 = ib_impact + control_variables + learning_variables + Y2009_drivers[:-1]
m4_inDepV_Y2010 = ib_impact + control_variables + learning_variables + Y2010_drivers[:-1]

In [17]:
# M4
dataset = Y2009_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m4_inDepV_Y2009})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <1> + <10123> + <10128> + <10146> + <10171> + <10172> + <1022>
             + <10288> + <10534> + <10602> + <10604> + <10626> + <10648> + <10677>
             + <1079> + <10850> + <10904> + <10969> + <1098> + <10990> + <11017>
             + <1108> + <11435> + <11587> + <11805> + <11845> + <11897> + <11911>
             + <1198> + <12044> + <12047> + <12090> + <1213> + <12173> + <12185>
             + <12304> + <12357> + <12392> + <12443> + <12494> + <12530> + <12583>
             + <12590> + <12900> + <12929> + <1295> + <12953> + <13041>
             + <13147> + <1318> + <1335> + <13457> + <13468> + <13508> + <13539>
             + <13612> + <13617> + <13692> + <13701> + <13733> + <1378> + <13799>
             + <14023> + <1406> + <14088> + <1418> + <14186> + <14291> + <1432>
             + <14372> + <14452> + <14541> + <14603> + <14613> + <14653> + <14694>
             + <14829> + <14878> 

In [18]:
# M4
dataset = Y2010_df
#
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m4_inDepV_Y2010})
model


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <10072> + <10073> + <10108> + <10123> + <10128> + <10146> + <10171>
             + <10172> + <1022> + <10322> + <10370> + <1042> + <10452> + <10453>
             + <10456> + <10534> + <10556> + <10602> + <10604> + <10623>
             + <10636> + <10648> + <10677> + <10728> + <10770> + <1078> + <1079>
             + <10850> + <10904> + <10969> + <10990> + <10996> + <11007> + <1108>
             + <11340> + <11516> + <11587> + <11588> + <1176> + <11805> + <11830>
             + <11929> + <11933> + <1198> + <11988> + <12047> + <12055> + <12086>
             + <12110> + <1213> + <12185> + <12304> + <12357> + <12392> + <124>
             + <12443> + <12450> + <12494> + <12530> + <12590> + <12617> + <1267>
             + <1274> + <12900> + <12920> + <12929> + <1299> + <12998> + <13041>
             + <1307> + <13120> + <13147> + <13148> + <1318> + <13468> + <13489>
             + <13508> + <13538

In [20]:
m5_inDepV, m5_monthFE, m5_driverFE = ib_impact + control_variables, True, True

items = [cn for cn in Y2009_df.columns if cn not in ['month', 'driverID'] + Y2009_drivers + Y2010_drivers]

Y2009_pd = pd.Panel(dict(zip(items, [Y2009_df.pivot(index='month', columns='driverID', values=i) for i in items])))
Y2010_pd = pd.Panel(dict(zip(items, [Y2010_df.pivot(index='month', columns='driverID', values=i) for i in items])))

In [22]:
# M5
dataset = Y2009_pd
#
print dataset
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m5_inDepV},
               entity_effects=m5_monthFE,
              time_effects=m5_driverFE,
              )
model

<class 'pandas.core.panel.Panel'>
Dimensions: 16 (items) x 11 (major_axis) x 597 (minor_axis)
Items axis: Fare to tripNumber
Major_axis axis: 1 to 11
Minor_axis axis: 1 to 38107


/Users/JerryHan88/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.plm module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://statsmodels.sourceforge.net/stable/mixed_linear.html
  exec(code_obj, self.user_global_ns, self.user_ns)



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <apProductivity>
             + <economicProfit/apTrip> + <tripNumber> + <FE_169> + <FE_199> + <FE_209>
             + <FE_229> + <FE_298> + <FE_362> + <FE_366> + <FE_382> + <FE_532>
             + <FE_562> + <FE_590> + <FE_591> + <FE_714> + <FE_716> + <FE_775>
             + <FE_795> + <FE_813> + <FE_836> + <FE_882> + <FE_889> + <FE_950>
             + <FE_1022> + <FE_1079> + <FE_1098> + <FE_1108> + <FE_1198> + <FE_1213>
             + <FE_1295> + <FE_1318> + <FE_1335> + <FE_1378> + <FE_1406>
             + <FE_1418> + <FE_1432> + <FE_1580> + <FE_1631> + <FE_1740> + <FE_1784>
             + <FE_1802> + <FE_1831> + <FE_2020> + <FE_2157> + <FE_2311> + <FE_2367>
             + <FE_2439> + <FE_2580> + <FE_2682> + <FE_2804> + <FE_2829>
             + <FE_2837> + <FE_2860> + <FE_2959> + <FE_3003> + <FE_3023> + <FE_3035>
             + <FE_3060> + <FE_31

In [23]:
# M5
dataset = Y2010_pd
#
print dataset
model = pd.ols(y=dataset[dep_v], 
               x={v: dataset[v] for v in m5_inDepV},
               entity_effects=m5_monthFE,
              time_effects=m5_driverFE,
              )
model

<class 'pandas.core.panel.Panel'>
Dimensions: 16 (items) x 11 (major_axis) x 752 (minor_axis)
Items axis: Fare to tripNumber
Major_axis axis: 1 to 12
Minor_axis axis: 87 to 49770



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Productivity> + <apInRatio> + <apNumber> + <apProductivity>
             + <economicProfit/apTrip> + <tripNumber> + <FE_124> + <FE_169> + <FE_174>
             + <FE_199> + <FE_209> + <FE_229> + <FE_253> + <FE_298> + <FE_358>
             + <FE_366> + <FE_382> + <FE_462> + <FE_464> + <FE_481> + <FE_553>
             + <FE_590> + <FE_714> + <FE_716> + <FE_732> + <FE_735> + <FE_762>
             + <FE_775> + <FE_795> + <FE_807> + <FE_813> + <FE_833> + <FE_836>
             + <FE_889> + <FE_933> + <FE_980> + <FE_1022> + <FE_1042> + <FE_1078>
             + <FE_1079> + <FE_1108> + <FE_1176> + <FE_1198> + <FE_1213> + <FE_1267>
             + <FE_1274> + <FE_1299> + <FE_1307> + <FE_1318> + <FE_1378> + <FE_1406>
             + <FE_1432> + <FE_1483> + <FE_1580> + <FE_1631> + <FE_1730>
             + <FE_1740> + <FE_1784> + <FE_1802> + <FE_1831> + <FE_1919> + <FE_2016>
             + <FE_2126> + <FE